In [ ]:
import pandas as pd
import earthaccess
import geopandas as gpd
import xarray as xr
import numpy as np
import geopandas as gpd
from shapely.geometry import Point, box
import pandas as pd
import geopandas as gpd
from geopy.geocoders import Nominatim
from shapely.geometry import shape
import time
import folium
import geopandas as gpd
import math
from shapely.geometry import Polygon, Point, LineString
import h5py

floripa_bounds = (-48.7242431, -27.917059, -48.3585929, -27.3804967)
floripa_bounds_polygon = box(*floripa_bounds)
floripa_center = [-27.5935, -48.5585]
dates = [("2025-01-23", "2025-01-23"), ("2025-01-30", "2025-01-30"), ("2025-02-06", "2025-02-06")]

def truncar(numero, casas_decimais):
    """Trunca um número para um número específico de casas decimais."""
    fator = 10.0 ** casas_decimais
    return math.trunc(numero * fator) / fator

def truncar_geometria(geom, casas_decimais):
    """
    Função inteligente que verifica o tipo de geometria e aplica o método
    de truncamento correto para Polígonos, Linhas e Pontos.
    """
    if geom is None or geom.is_empty:
        return None

    # Se for um Polígono, usa a lógica de exterior/interiores
    if geom.geom_type == 'Polygon':
        coords_exteriores = [(truncar(x, casas_decimais), truncar(y, casas_decimais)) for x, y in geom.exterior.coords]
        coords_interiores = []
        for interior in geom.interiors:
            coords_interiores.append([(truncar(x, casas_decimais), truncar(y, casas_decimais)) for x, y in interior.coords])
        return Polygon(coords_exteriores, coords_interiores)

    # Se for uma Linha, usa a lógica simples de .coords
    elif geom.geom_type == 'LineString':
        coords = [(truncar(x, casas_decimais), truncar(y, casas_decimais)) for x, y in geom.coords]
        return LineString(coords)

    # Se for um Ponto, usa a lógica de .x e .y
    elif geom.geom_type == 'Point':
        return Point(truncar(geom.x, casas_decimais), truncar(geom.y, casas_decimais))
        
    # Para outros tipos de geometria (MultiPolygon, etc.), retorna o original
    else:
        print(f"Aviso: Tipo de geometria '{geom.geom_type}' não suportado para truncamento. Retornando original.")
        return geom

def get_districts_loction(df_districts):
    geolocator = Nominatim(user_agent="busca_bairros_floripa_app")
    geometrias = []

    print(f"Iniciando a busca de geometrias para {len(df_districts)} bairros. Isso pode levar um minuto...")

    for bairro in df_districts['Bairro']:
        try:
            query = f"{bairro}, Florianópolis, Santa Catarina"
            location = geolocator.geocode(query, geometry='geojson')
            
            if location and 'geojson' in location.raw:
                geom = shape(location.raw['geojson'])
                geometrias.append(geom)
            else:
                geometrias.append(None)
        except Exception as e:
            geometrias.append(None)
        time.sleep(1)

    df_districts['geometry'] = geometrias
    bairros_gdf = gpd.GeoDataFrame(df_districts, geometry='geometry', crs="EPSG:4326")
    bairros_gdf.dropna(subset=['geometry'], inplace=True)

    return bairros_gdf

def download_metric(metric, range_dates, bbox, files_qty=1):
    try:
        auth = earthaccess.login(strategy="netrc")
        if not auth.authenticated:
            print("Autenticação falhou! Verifique seu arquivo .netrc.")
            return
    except Exception as e:
        print(f"Erro na autenticação: {e}")
        return
    
    # Procurando os dados GPM IMERG Half-Hourly
    # Short Name para o produto IMERG Final Run Half-Hourly V07: GPM_3IMERGHH
    results = earthaccess.search_data(
        short_name=metric,
        bounding_box=bbox,
        temporal=range_dates, # Exemplo: buscando 1 dia de dados
        count=files_qty
    )
    if not results:
        print("pesquisa sem resultados")
        return
    
    d_files = []
    for r in results:
        d_files.append(earthaccess.download(r, local_path="data"))
    return d_files

def merge_with_smap(smap_gdf: gpd.GeoDataFrame, bairros_gdf: gpd.GeoDataFrame, max_distance=5000) -> pd.DataFrame:
    df_final = pd.DataFrame()
    for date in smap_gdf['date'].unique():
        gpm_on_date = smap_gdf[smap_gdf['date'] == date]
        
        bairros_com_chuva = gpd.sjoin_nearest(bairros_gdf, gpm_on_date, how="left", max_distance=max_distance)

        df_final = pd.concat([df_final, bairros_com_chuva], ignore_index=True)
    
    df_final = df_final.sort_values(by="soil_moisture", ascending=False)
    df_final = df_final.drop_duplicates(subset=["Bairro"], keep="first")

    df_final = df_final.replace(np.nan, 0)
    df_final.reset_index(drop=True, inplace=True)
    df_final.drop("index_right", axis=1, inplace=True)
    return df_final

def merge_with_precipitation(df_gpm: gpd.GeoDataFrame, df_districts: gpd.GeoDataFrame, max_distance=5000):
    df_final = pd.DataFrame()
    for date in df_gpm['date'].unique():
        gpm_on_date = df_gpm[df_gpm['date'] == date]
        bairros_com_chuva = gpd.sjoin_nearest(df_districts, gpm_on_date, how="left", max_distance=max_distance)

        df_final = pd.concat([df_final, bairros_com_chuva], ignore_index=True)
    
    df_final = df_final.sort_values(by="precipitation_mmhr", ascending=False)
    df_final = df_final.drop_duplicates(subset=["Bairro"], keep="first")

    df_final = df_final.replace(np.nan, 0)
    df_final.reset_index(drop=True, inplace=True)
    df_final.drop("index_right", axis=1, inplace=True)
    return df_final

def generate_html_graph(data, center, map_name):
    m = folium.Map(location=center, zoom_start=11)

    folium.GeoJson(
        data,
        name=map_name,
        tooltip=folium.GeoJsonTooltip(fields=['Bairro'], aliases=['Bairro:']),
        style_function=lambda x: {
            'fillColor': '#4CAF50', # Cor de preenchimento verde
            'color': 'black',      # Cor da borda preta
            'weight': 1,           # Espessura da borda
            'fillOpacity': 0.4     # Opacidade do preenchimento
        }
    ).add_to(m)

    folium.LayerControl().add_to(m)

    mapa_path = f'{map_name}.html'
    m.save(mapa_path)

def get_gpm(ds):
    precip_data = ds['precipitation'].values[0]
    lats = ds['lat'].values
    lons = ds['lon'].values
    
    lon_grid, lat_grid = np.meshgrid(lons, lats)
    points = [Point(lon, lat) for lon, lat in zip(lon_grid.ravel(), lat_grid.ravel())]
    precip_values = precip_data.ravel()

    gpm_gdf = gpd.GeoDataFrame({
        'precipitation_mmhr': precip_values,
        'geometry': points
    }, crs="EPSG:4326")
    
    return gpm_gdf


def safe_sjoin_nearest(district_group, smap_grouped):
    try:
        smap_group = smap_grouped.get_group(district_group.name)

        smap_gdf = gpd.GeoDataFrame(smap_group, geometry='geometry', crs="EPSG:4326")

        return gpd.sjoin_nearest(district_group, smap_gdf, how="left")

    except KeyError:
        # Se a chave (data) não for encontrada, este bloco é executado
        result = district_group.copy()

        # Adiciona as colunas que viriam do SMAP e preenche com 0
        # sjoin_nearest também adiciona 'index_right', então incluímos ela.
        cols_from_smap = ['index_right', 'soil_moisture']
        for col in cols_from_smap:
            result[col] = 0

        return result

In [38]:
df_floripa_districts = pd.read_csv(r"data\floripa_districts.csv")
df_floripa_districts = get_districts_loction(df_floripa_districts)

Iniciando a busca de geometrias para 49 bairros. Isso pode levar um minuto...


In [ ]:
files_qty = 10
dfs_gpm = []
df_gpm_final = pd.DataFrame()
df_smap_final = pd.DataFrame()
for date in dates:
    gpm_f = download_metric("GPM_3IMERGHHL", date, floripa_bounds, files_qty=files_qty)
    for f in gpm_f:
        f = f[0]
        print(f"Processing {f}...")
        try:
            ds = xr.open_dataset(f, group='Grid', engine="h5netcdf")
        except Exception as e:
            print(f"Erro ao abrir o arquivo {f}: {e}")
            continue
        ds = get_gpm(ds)
        name = f.replace(".HDF5", ".csv")
        limits = ds.geometry.within(floripa_bounds_polygon)
        ds = ds[limits]
        ds["date"] = date[0]
        ds.to_csv(name, index=False)
        dfs_gpm.append(name)

    df_gpm = pd.DataFrame()
    for gpm in dfs_gpm:
        df_temp = pd.read_csv(gpm)
        df_gpm = pd.concat([df_gpm, df_temp], ignore_index=True)
    df_gpm_final = pd.concat([df_gpm_final, df_gpm], ignore_index=True)

smap_dates = [("2025-01-23", "2025-01-24"), ("2025-01-30", "2025-01-31"), ("2025-02-06", "2025-02-07")]
for date in smap_dates:
    smap_f = download_metric("SPL3SMP_E", date, floripa_bounds, files_qty=files_qty)
    dfs_smap = []
    for f in smap_f:
        f = f[0]
        print(f"Processing {f}...")
        with h5py.File(f, 'r') as hf:
            data_group = hf['Soil_Moisture_Retrieval_Data_AM']
            
            moisture_dataset = data_group['soil_moisture']
            lats_dataset = data_group['latitude']
            lons_dataset = data_group['longitude']
            
            moisture_data = moisture_dataset[:]
            lats = lats_dataset[:]
            lons = lons_dataset[:]
            
            fill_value = moisture_dataset.attrs.get('_FillValue', 0)

        moisture_values_flat = moisture_data.ravel()
        
        lons_flat = lons.ravel()
        lats_flat = lats.ravel()

        valid_mask = moisture_values_flat != fill_value
        points = [Point(lon, lat) for lon, lat in zip(lons_flat[valid_mask], lats_flat[valid_mask])]
        valid_moisture_values = moisture_values_flat[valid_mask]

        smap_gdf = gpd.GeoDataFrame({
            'soil_moisture': valid_moisture_values,
            'geometry': points
        }, crs="EPSG:4326")
        
        name = f.replace(".h5", ".csv")
        limits = smap_gdf.geometry.within(floripa_bounds_polygon)
        smap_gdf = smap_gdf[limits]
        smap_gdf["date"] = date[0]
        smap_gdf.to_csv(name, index=False)
        dfs_smap.append(name)

    df_smap = pd.DataFrame()
    for smap in dfs_smap:
        df_temp = pd.read_csv(smap)
        df_smap = pd.concat([df_smap, df_temp], ignore_index=True)
    df_smap_final = pd.concat([df_smap_final, df_smap], ignore_index=True)
    

df_gpm_final.to_csv(f"data/gpm.csv", index=False)
df_smap_final.to_csv(f"data/smap.csv", index=False)

smap = gpd.GeoDataFrame(
        df_smap_final,
        geometry=gpd.GeoSeries.from_wkt(df_smap_final['geometry']),
        crs="EPSG:4326"
    )
gpm = gpd.GeoDataFrame(
        df_gpm_final,
        geometry=gpd.GeoSeries.from_wkt(df_gpm['geometry']),
        crs="EPSG:4326"
    )

QUEUEING TASKS | : 100%|██████████| 1/1 [00:00<00:00, 999.83it/s]
PROCESSING TASKS | : 100%|██████████| 1/1 [00:00<?, ?it/s]
COLLECTING RESULTS | : 100%|██████████| 1/1 [00:00<00:00, 1000.31it/s]
QUEUEING TASKS | : 100%|██████████| 1/1 [00:00<?, ?it/s]
PROCESSING TASKS | : 100%|██████████| 1/1 [00:04<00:00,  4.48s/it]
COLLECTING RESULTS | : 100%|██████████| 1/1 [00:00<?, ?it/s]
QUEUEING TASKS | : 100%|██████████| 1/1 [00:00<00:00, 500.22it/s]
PROCESSING TASKS | : 100%|██████████| 1/1 [00:04<00:00,  4.46s/it]
COLLECTING RESULTS | : 100%|██████████| 1/1 [00:00<?, ?it/s]
QUEUEING TASKS | : 100%|██████████| 1/1 [00:00<00:00, 1000.55it/s]
PROCESSING TASKS | : 100%|██████████| 1/1 [00:04<00:00,  4.76s/it]
COLLECTING RESULTS | : 100%|██████████| 1/1 [00:00<?, ?it/s]
QUEUEING TASKS | : 100%|██████████| 1/1 [00:00<00:00, 1001.51it/s]
PROCESSING TASKS | : 100%|██████████| 1/1 [00:04<00:00,  4.46s/it]
COLLECTING RESULTS | : 100%|██████████| 1/1 [00:00<?, ?it/s]
QUEUEING TASKS | : 100%|██████████|

Processing data\3B-HHR-L.MS.MRG.3IMERG.20250123-S000000-E002959.0000.V07B.HDF5...
Processing data\3B-HHR-L.MS.MRG.3IMERG.20250123-S003000-E005959.0030.V07B.HDF5...
Processing data\3B-HHR-L.MS.MRG.3IMERG.20250123-S010000-E012959.0060.V07B.HDF5...
Processing data\3B-HHR-L.MS.MRG.3IMERG.20250123-S013000-E015959.0090.V07B.HDF5...
Processing data\3B-HHR-L.MS.MRG.3IMERG.20250123-S020000-E022959.0120.V07B.HDF5...
Processing data\3B-HHR-L.MS.MRG.3IMERG.20250123-S023000-E025959.0150.V07B.HDF5...
Processing data\3B-HHR-L.MS.MRG.3IMERG.20250123-S030000-E032959.0180.V07B.HDF5...
Processing data\3B-HHR-L.MS.MRG.3IMERG.20250123-S033000-E035959.0210.V07B.HDF5...
Processing data\3B-HHR-L.MS.MRG.3IMERG.20250123-S040000-E042959.0240.V07B.HDF5...
Processing data\3B-HHR-L.MS.MRG.3IMERG.20250123-S043000-E045959.0270.V07B.HDF5...


QUEUEING TASKS | : 100%|██████████| 1/1 [00:00<00:00, 999.60it/s]
PROCESSING TASKS | : 100%|██████████| 1/1 [00:24<00:00, 24.66s/it]
COLLECTING RESULTS | : 100%|██████████| 1/1 [00:00<?, ?it/s]
QUEUEING TASKS | : 100%|██████████| 1/1 [00:00<?, ?it/s]
PROCESSING TASKS | : 100%|██████████| 1/1 [00:00<?, ?it/s]
COLLECTING RESULTS | : 100%|██████████| 1/1 [00:00<?, ?it/s]


Processing data\SMAP_L3_SM_P_E_20250123_R19240_002.h5...
Processing data\SMAP_L3_SM_P_E_20250123_R19240_002.h5...


QUEUEING TASKS | : 100%|██████████| 1/1 [00:00<00:00, 1607.01it/s]
PROCESSING TASKS | : 100%|██████████| 1/1 [00:04<00:00,  4.88s/it]
COLLECTING RESULTS | : 100%|██████████| 1/1 [00:00<?, ?it/s]
QUEUEING TASKS | : 100%|██████████| 1/1 [00:00<?, ?it/s]
PROCESSING TASKS | : 100%|██████████| 1/1 [00:04<00:00,  4.77s/it]
COLLECTING RESULTS | : 100%|██████████| 1/1 [00:00<?, ?it/s]
QUEUEING TASKS | : 100%|██████████| 1/1 [00:00<00:00, 999.83it/s]
PROCESSING TASKS | : 100%|██████████| 1/1 [00:04<00:00,  4.56s/it]
COLLECTING RESULTS | : 100%|██████████| 1/1 [00:00<?, ?it/s]
QUEUEING TASKS | : 100%|██████████| 1/1 [00:00<?, ?it/s]
PROCESSING TASKS | : 100%|██████████| 1/1 [00:04<00:00,  4.38s/it]
COLLECTING RESULTS | : 100%|██████████| 1/1 [00:00<?, ?it/s]
QUEUEING TASKS | : 100%|██████████| 1/1 [00:00<00:00, 999.36it/s]
PROCESSING TASKS | : 100%|██████████| 1/1 [00:04<00:00,  4.36s/it]
COLLECTING RESULTS | : 100%|██████████| 1/1 [00:00<?, ?it/s]
QUEUEING TASKS | : 100%|██████████| 1/1 [00:00<

...

C:\Users\Bee\AppData\Local\Temp\ipykernel_7964\2607836854.py:72: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_smap_final = pd.concat([df_smap_final, df_smap], ignore_index=True)
QUEUEING TASKS | : 100%|██████████| 1/1 [00:00<00:00, 1781.02it/s]
PROCESSING TASKS | : 100%|██████████| 1/1 [00:04<00:00,  4.75s/it]
COLLECTING RESULTS | : 100%|██████████| 1/1 [00:00<?, ?it/s]
QUEUEING TASKS | : 100%|██████████| 1/1 [00:00<00:00, 998.64it/s]
PROCESSING TASKS | : 100%|██████████| 1/1 [00:04<00:00,  4.30s/it]
COLLECTING RESULTS | : 100%|██████████| 1/1 [00:00<?, ?it/s]
QUEUEING TASKS | : 100%|██████████| 1/1 [00:00<00:00, 999.60it/s]
PROCESSING TASKS | : 100%|██████████| 1/1 [00:04<00:00,  4.57s/it]
COLLECTING RESULTS | : 100%|██████████| 1/1 [0

Processing data\3B-HHR-L.MS.MRG.3IMERG.20250206-S000000-E002959.0000.V07B.HDF5...
Processing data\3B-HHR-L.MS.MRG.3IMERG.20250206-S003000-E005959.0030.V07B.HDF5...
Processing data\3B-HHR-L.MS.MRG.3IMERG.20250206-S010000-E012959.0060.V07B.HDF5...
Processing data\3B-HHR-L.MS.MRG.3IMERG.20250206-S013000-E015959.0090.V07B.HDF5...
Processing data\3B-HHR-L.MS.MRG.3IMERG.20250206-S020000-E022959.0120.V07B.HDF5...
Processing data\3B-HHR-L.MS.MRG.3IMERG.20250206-S023000-E025959.0150.V07B.HDF5...
Processing data\3B-HHR-L.MS.MRG.3IMERG.20250206-S030000-E032959.0180.V07B.HDF5...
Processing data\3B-HHR-L.MS.MRG.3IMERG.20250206-S033000-E035959.0210.V07B.HDF5...
Processing data\3B-HHR-L.MS.MRG.3IMERG.20250206-S040000-E042959.0240.V07B.HDF5...
Processing data\3B-HHR-L.MS.MRG.3IMERG.20250206-S043000-E045959.0270.V07B.HDF5...


QUEUEING TASKS | : 100%|██████████| 1/1 [00:00<00:00, 1004.14it/s]
PROCESSING TASKS | : 100%|██████████| 1/1 [00:48<00:00, 48.79s/it]
COLLECTING RESULTS | : 100%|██████████| 1/1 [00:00<?, ?it/s]
QUEUEING TASKS | : 100%|██████████| 1/1 [00:00<00:00, 983.65it/s]
PROCESSING TASKS | : 100%|██████████| 1/1 [00:00<00:00, 255.30it/s]
COLLECTING RESULTS | : 100%|██████████| 1/1 [00:00<00:00, 993.91it/s]


Processing data\SMAP_L3_SM_P_E_20250206_R19240_001.h5...
Processing data\SMAP_L3_SM_P_E_20250206_R19240_001.h5...


C:\Users\Bee\AppData\Local\Temp\ipykernel_7964\2607836854.py:72: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_smap_final = pd.concat([df_smap_final, df_smap], ignore_index=True)


In [74]:
smap_dates = [("2025-01-23", "2025-01-24"), ("2025-01-30", "2025-01-31"), ("2025-02-06", "2025-02-07")]
for date in smap_dates:
    smap_f = download_metric("SPL3SMP_E", date, floripa_bounds, files_qty=files_qty)
    dfs_smap = []
    for f in smap_f:
        f = f[0]
        print(f"Processing {f}...")
        with h5py.File(f, 'r') as hf:
            data_group = hf['Soil_Moisture_Retrieval_Data_AM']
            
            moisture_dataset = data_group['soil_moisture']
            lats_dataset = data_group['latitude']
            lons_dataset = data_group['longitude']
            
            moisture_data = moisture_dataset[:]
            lats = lats_dataset[:]
            lons = lons_dataset[:]
            
            fill_value = moisture_dataset.attrs.get('_FillValue', 0)

        moisture_values_flat = moisture_data.ravel()
        
        lons_flat = lons.ravel()
        lats_flat = lats.ravel()

        valid_mask = moisture_values_flat != fill_value
        points = [Point(lon, lat) for lon, lat in zip(lons_flat[valid_mask], lats_flat[valid_mask])]
        valid_moisture_values = moisture_values_flat[valid_mask]

        smap_gdf = gpd.GeoDataFrame({
            'soil_moisture': valid_moisture_values,
            'geometry': points
        }, crs="EPSG:4326")
        
        name = f.replace(".h5", ".csv")
        limits = smap_gdf.geometry.within(floripa_bounds_polygon)
        smap_gdf = smap_gdf[limits]
        smap_gdf["date"] = date[0]
        smap_gdf.to_csv(name, index=False)
        dfs_smap.append(name)

    df_smap = pd.DataFrame()
    for smap in dfs_smap:
        df_temp = pd.read_csv(smap)
        df_smap = pd.concat([df_smap, df_temp], ignore_index=True)
    df_smap_final = pd.concat([df_smap_final, df_smap], ignore_index=True)
    

df_smap_final.to_csv(f"data/smap.csv", index=False)

smap = gpd.GeoDataFrame(
        df_smap_final,
        geometry=gpd.GeoSeries.from_wkt(df_smap_final['geometry']),
        crs="EPSG:4326"
    )


QUEUEING TASKS | : 100%|██████████| 1/1 [00:00<?, ?it/s]
PROCESSING TASKS | : 100%|██████████| 1/1 [00:00<?, ?it/s]
COLLECTING RESULTS | : 100%|██████████| 1/1 [00:00<?, ?it/s]
QUEUEING TASKS | : 100%|██████████| 1/1 [00:00<00:00, 999.83it/s]
PROCESSING TASKS | : 100%|██████████| 1/1 [00:29<00:00, 29.33s/it]
COLLECTING RESULTS | : 100%|██████████| 1/1 [00:00<?, ?it/s]
QUEUEING TASKS | : 100%|██████████| 1/1 [00:00<?, ?it/s]
PROCESSING TASKS | : 100%|██████████| 1/1 [00:00<?, ?it/s]
COLLECTING RESULTS | : 100%|██████████| 1/1 [00:00<?, ?it/s]
QUEUEING TASKS | : 100%|██████████| 1/1 [00:00<?, ?it/s]
PROCESSING TASKS | : 100%|██████████| 1/1 [00:00<?, ?it/s]
COLLECTING RESULTS | : 100%|██████████| 1/1 [00:00<?, ?it/s]


Processing data\SMAP_L3_SM_P_E_20250123_R19240_002.h5...
Processing data\SMAP_L3_SM_P_E_20250124_R19240_002.h5...
Processing data\SMAP_L3_SM_P_E_20250123_R19240_002.h5...
Processing data\SMAP_L3_SM_P_E_20250124_R19240_002.h5...


C:\Users\Bee\AppData\Local\Temp\ipykernel_7964\3173943724.py:45: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_smap = pd.concat([df_smap, df_temp], ignore_index=True)
C:\Users\Bee\AppData\Local\Temp\ipykernel_7964\3173943724.py:45: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_smap = pd.concat([df_smap, df_temp], ignore_index=True)
QUEUEING TASKS | : 100%|██████████| 1/1 [00:00<00:00, 1000.55it/s]
PROCESSING TASKS | : 100%|██████████| 1/1 [00:00<?, ?it/s]
COLLECTING RESULTS | : 100%|██████

Processing data\SMAP_L3_SM_P_E_20250130_R19240_001.h5...
Processing data\SMAP_L3_SM_P_E_20250131_R19240_002.h5...
Processing data\SMAP_L3_SM_P_E_20250130_R19240_001.h5...
Processing data\SMAP_L3_SM_P_E_20250131_R19240_002.h5...


C:\Users\Bee\AppData\Local\Temp\ipykernel_7964\3173943724.py:45: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_smap = pd.concat([df_smap, df_temp], ignore_index=True)
C:\Users\Bee\AppData\Local\Temp\ipykernel_7964\3173943724.py:45: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_smap = pd.concat([df_smap, df_temp], ignore_index=True)
QUEUEING TASKS | : 100%|██████████| 1/1 [00:00<00:00, 995.33it/s]
PROCESSING TASKS | : 100%|██████████| 1/1 [00:00<?, ?it/s]
COLLECTING RESULTS | : 100%|███████

Processing data\SMAP_L3_SM_P_E_20250206_R19240_001.h5...
Processing data\SMAP_L3_SM_P_E_20250207_R19240_002.h5...
Processing data\SMAP_L3_SM_P_E_20250206_R19240_001.h5...
Processing data\SMAP_L3_SM_P_E_20250207_R19240_002.h5...


C:\Users\Bee\AppData\Local\Temp\ipykernel_7964\3173943724.py:46: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_smap_final = pd.concat([df_smap_final, df_smap], ignore_index=True)


In [191]:
df_floripa_districts = merge_with_smap(smap, df_floripa_districts)

d:\Users\Bee\Estudos\nasa-challenge\nasa-challenge\lib\site-packages\geopandas\array.py:408: UserWarning: Geometry is in a geographic CRS. Results from 'sjoin_nearest' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  warnings.warn(
d:\Users\Bee\Estudos\nasa-challenge\nasa-challenge\lib\site-packages\geopandas\array.py:408: UserWarning: Geometry is in a geographic CRS. Results from 'sjoin_nearest' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  warnings.warn(


In [194]:
df_floripa_districts

,Bairro,geometry,soil_moisture,date
0,Caieira da Barra do Sul,"POLYGON ((-48.56401 -27.81713, -48.56397 -27.8...",0.485102,2025-01-30
1,Ribeirão da Ilha,"POLYGON ((-48.58158 -27.76212, -48.58156 -27.7...",0.392678,2025-01-30
2,Tapera,POINT (-48.56128 -27.78806),0.392678,2025-01-30
3,Pântano do Sul,"POLYGON ((-48.55926 -27.76745, -48.55876 -27.7...",0.392678,2025-01-30
4,Rio Vermelho,"LINESTRING (-48.75238 -27.79113, -48.75248 -27...",0.392678,2025-01-30
5,Armação do Pântano do Sul,"POLYGON ((-48.53388 -27.74716, -48.53331 -27.7...",0.392678,2025-01-30
6,Cachoeira do Bom Jesus,"POLYGON ((-48.45818 -27.46639, -48.4577 -27.46...",0.376195,2025-01-30
7,Jurerê Internacional,"POLYGON ((-48.52374 -27.45185, -48.52362 -27.4...",0.376195,2025-01-30
8,Santo Antônio de Lisboa,"POLYGON ((-48.53954 -27.48683, -48.53953 -27.4...",0.376195,2025-01-30
9,Sambaqui,"POLYGON ((-48.53954 -27.48683, -48.53953 -27.4...",0.376195,2025-01-30


In [ ]:
df_floripa_districts = merge_with_smap(smap, df_floripa_districts)
df_floripa_districts = merge_with_precipitation(gpm, df_floripa_districts)

df_srtm = pd.read_csv("data/srtm.csv")
df_srtm = df_srtm[["Bairro", "Inclinacao_Max"]].rename(columns={"Inclinacao_Max": "srtm_max"})

df_floripa_districts = df_floripa_districts.merge(df_srtm, on="Bairro", how="left")
df_floripa_districts["srtm_max"] = df_floripa_districts["srtm_max"].fillna(0)
df_floripa_districts.rename(columns={"date_left": "date"}, inplace=True)

d:\Users\Bee\Estudos\nasa-challenge\nasa-challenge\lib\site-packages\geopandas\array.py:408: UserWarning: Geometry is in a geographic CRS. Results from 'sjoin_nearest' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  warnings.warn(
d:\Users\Bee\Estudos\nasa-challenge\nasa-challenge\lib\site-packages\geopandas\array.py:408: UserWarning: Geometry is in a geographic CRS. Results from 'sjoin_nearest' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  warnings.warn(
d:\Users\Bee\Estudos\nasa-challenge\nasa-challenge\lib\site-packages\geopandas\array.py:408: UserWarning: Geometry is in a geographic CRS. Results from 'sjoin_nearest' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  warnings.warn(
d:\Users\Bee\Estudos\nasa-challenge\nasa-challenge\lib\site-packages\geopandas\array.py:408: UserWar

In [184]:
def gpm_score(s):
    if s['precipitation_mmhr'] <= 20:
        return 'Baixa'
    elif s['precipitation_mmhr'] <= 50:
        return 'Moderada'
    elif s['precipitation_mmhr'] <= 100:
        return 'Alta'
    else:
        return 'Crítica'
    
def smap_score(s):
    if s['soil_moisture'] <= 0.15:
        return 'Baixa'
    elif s['soil_moisture'] <= 0.30:
        return 'Moderada'
    elif s['soil_moisture'] <= .40:
        return 'Alta'
    else:
        return 'Crítica'
    
def srtm_score(s):
    if s['srtm_max'] <= 5:
        return 'Baixa'
    elif s['srtm_max'] <= 15:
        return 'Moderada'
    elif s['srtm_max'] <= 30:
        return 'Alta'
    else:
        return 'Crítica'
    
df_floripa_districts["gpm_score"] = df_floripa_districts.apply(gpm_score, axis=1)
df_floripa_districts["smap_score"] = df_floripa_districts.apply(smap_score, axis=1)
df_floripa_districts["srtm_score"] = df_floripa_districts.apply(srtm_score, axis=1)

In [185]:
from itertools import product

niveis = ['Baixa', 'Média', 'Alta', 'Crítica']

combinacoes = list(product(niveis, repeat=3))

df_matrix = pd.DataFrame(combinacoes, columns=['srtm_score', 'gpm_score', 'smap_score'])

score_map = {'Baixa': 1, 'Média': 2, 'Alta': 3, 'Crítica': 4}

def definir_risco_e_acao(row):
    """Calcula o risco e a ação com base na soma dos scores."""
    total_score = score_map[row['srtm_score']] + \
                  score_map[row['gpm_score']] + \
                  score_map[row['smap_score']]
    
    if total_score <= 5:
        return 'Baixo', 'Observar'
    elif total_score <= 7:
        return 'Moderado', 'Monitorar'
    elif total_score <= 9:
        return 'Alto', 'Alertar'
    else: # total_score >= 10
        return 'Crítico', 'Evacuar'

df_matrix[['risk_classification', 'recommended_action']] = df_matrix.apply(
    definir_risco_e_acao, axis=1, result_type='expand'
)

df_floripa_districts = pd.merge(
    df_floripa_districts,      # O DataFrame da esquerda (principal)
    df_matrix,                 # O DataFrame da direita (tabela de consulta)
    on=['srtm_score', 'gpm_score', 'smap_score'],  # As colunas-chave para a correspondência
    how='left'                 # Tipo de junção: mantém tudo da esquerda
)

df_floripa_districts.loc[df_floripa_districts.risk_classification.isna(), "risk_classification"] = "Moderado"
df_floripa_districts.loc[df_floripa_districts.risk_classification.isna(), "recommended_action"] = "Monitorar"

df_floripa_districts["lat"] = df_floripa_districts.geometry.centroid.y
df_floripa_districts["lon"] = df_floripa_districts.geometry.centroid.x
df_floripa_districts.rename(columns={"date_left": "date"}, inplace=True)
df_floripa_districts.to_csv("data/data.csv", index=False)

C:\Users\Bee\AppData\Local\Temp\ipykernel_7964\1940871200.py:40: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  df_floripa_districts["lat"] = df_floripa_districts.geometry.centroid.y
C:\Users\Bee\AppData\Local\Temp\ipykernel_7964\1940871200.py:41: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  df_floripa_districts["lon"] = df_floripa_districts.geometry.centroid.x


In [188]:
df_floripa_districts = df_floripa_districts[["Bairro", "geometry"]]
df_floripa_districts.drop_duplicates(subset=["Bairro"], keep="first", inplace=True)
df_floripa_districts.reset_index(drop=True, inplace=True)

df_floripa_districts

In [189]:
df_floripa_districts

,Bairro,geometry
0,Campeche,"POLYGON ((-48.5245 -27.67651, -48.5245 -27.676..."
1,Caieira da Barra do Sul,"POLYGON ((-48.56401 -27.81713, -48.56397 -27.8..."
2,Morro das Pedras,"POLYGON ((-48.51527 -27.71372, -48.51201 -27.7..."
3,Pântano do Sul,"POLYGON ((-48.55926 -27.76745, -48.55876 -27.7..."
4,Tapera,POINT (-48.56128 -27.78806)
5,Armação do Pântano do Sul,"POLYGON ((-48.53388 -27.74716, -48.53331 -27.7..."
6,Jurerê,"POLYGON ((-48.51496 -27.43202, -48.51487 -27.4..."
7,Canasvieiras,"POLYGON ((-48.54722 -27.45903, -48.54715 -27.4..."
8,Cachoeira do Bom Jesus,"POLYGON ((-48.45818 -27.46639, -48.4577 -27.46..."
9,Ponta das Canas,"POLYGON ((-48.43663 -27.39291, -48.43661 -27.3..."


In [128]:
df_floripa_districts

,Bairro,geometry,index_right,precipitation_mmhr,date
0,Campeche,"POLYGON ((-48.5245 -27.67651, -48.5245 -27.676...",98,0.39,2025-01-23
20,Caieira da Barra do Sul,"POLYGON ((-48.56401 -27.81713, -48.56397 -27.8...",1,1.00,2025-01-23
40,Morro das Pedras,"POLYGON ((-48.51527 -27.71372, -48.51201 -27.7...",154,0.48,2025-01-23
60,Tapera,POINT (-48.56128 -27.78806),4,0.48,2025-01-23
80,Armação do Pântano do Sul,"POLYGON ((-48.53388 -27.74716, -48.53331 -27.7...",154,0.48,2025-01-23
...,...,...,...,...,...
1420,Santa Mônica,"POLYGON ((-48.5171 -27.58916, -48.51666 -27.58...",430,3.24,2025-01-30
1430,Rio Vermelho,"LINESTRING (-48.75238 -27.79113, -48.75248 -27...",423,7.08,2025-01-30
1440,Daniela,"POLYGON ((-48.54722 -27.45903, -48.54715 -27.4...",433,0.40,2025-01-30
1450,Jurerê Internacional,"POLYGON ((-48.52374 -27.45185, -48.52362 -27.4...",433,0.40,2025-01-30
